# Credit Risk Analysis

### NECESSARY IMPORTS

In [40]:
import boto3
import os
import sys
import numpy as np
import pandas as pd

# from dotenv import load_dotenv #for env variables
from imp import reload #for testing

from src import data_utils, config


### GET DATA

In [4]:
reload(data_utils)
reload(config)

# get data
app_train, app_test, columns_description = data_utils.get_datasets()


c:\Users\Usuario\OneDrive\Documents\GitHub\credit-risk-analysis\src\data_utils.py:51: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  app_train = pd.read_csv(config.DATASET_TRAIN, delimiter='\t', encoding='latin1', header=None)


In [5]:
app_train.head()

,ID_CLIENT,CLERK_TYPE,PAYMENT_DAY,APPLICATION_SUBMISSION_TYPE,QUANT_ADDITIONAL_CARDS,POSTAL_ADDRESS_TYPE,SEX,MARITAL_STATUS,QUANT_DEPENDANTS,EDUCATION_LEVEL,...,FLAG_HOME_ADDRESS_DOCUMENT,FLAG_RG,FLAG_CPF,FLAG_INCOME_PROOF,PRODUCT,FLAG_ACSP_RECORD,AGE,RESIDENCIAL_ZIP_3,PROFESSIONAL_ZIP_3,TARGET_LABEL_BAD=1
0,1,C,5,Web,0,1,F,6,1,0,...,0,0,0,0,1,N,32,595,595,1
1,2,C,15,Carga,0,1,F,2,0,0,...,0,0,0,0,1,N,34,230,230,1
2,3,C,5,Web,0,1,F,2,0,0,...,0,0,0,0,1,N,27,591,591,0
3,4,C,20,Web,0,1,F,2,0,0,...,0,0,0,0,1,N,61,545,545,0
4,5,C,10,Web,0,1,M,2,0,0,...,0,0,0,0,1,N,48,235,235,1


In [6]:
columns_description

,Var_Title,Var_Description,Field_Content
Var_Id,,,
1,ID_CLIENT,Sequential number for the applicant (to be use...,"1-50000, 50001-70000, 70001-90000"
2,CLERK_TYPE,Not informed,C
3,PAYMENT_DAY,"Day of the month for bill payment, chosen by t...","1,5,10,15,20,25"
4,APPLICATION_SUBMISSION_TYPE,Indicates if the application was submitted via...,"Web, Carga"
5,QUANT_ADDITIONAL_CARDS,Quantity of additional cards asked for in the ...,"1,2,NULL"
6,POSTAL_ADDRESS_TYPE,Indicates if the address for posting is the ho...,1.2
7,SEX,NaN,"M=Male, F=Female"
8,MARITAL_STATUS,Encoding not informed,"1,2,3,4,5,6,7"
9,QUANT_DEPENDANTS,NaN,"0, 1, 2, ..."


In [30]:
app_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 54 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID_CLIENT                       50000 non-null  int64  
 1   CLERK_TYPE                      50000 non-null  object 
 2   PAYMENT_DAY                     50000 non-null  int64  
 3   APPLICATION_SUBMISSION_TYPE     50000 non-null  object 
 4   QUANT_ADDITIONAL_CARDS          50000 non-null  int64  
 5   POSTAL_ADDRESS_TYPE             50000 non-null  int64  
 6   SEX                             50000 non-null  object 
 7   MARITAL_STATUS                  50000 non-null  int64  
 8   QUANT_DEPENDANTS                50000 non-null  int64  
 9   EDUCATION_LEVEL                 50000 non-null  int64  
 10  STATE_OF_BIRTH                  50000 non-null  object 
 11  CITY_OF_BIRTH                   50000 non-null  object 
 12  NACIONALITY                     

In [41]:
sex_values = np.unique(app_train["SEX"], return_counts=True)
print(dict(zip(sex_values[0], sex_values[1])))

#eliminar valores 'n' y ' '

{' ': 17, 'F': 30805, 'M': 19130, 'N': 48}


In [48]:
# Eliminar valores 'N' y ' '
# Reemplazar 'N' y ' ' por NaN en la columna "SEX"
app_train.loc[(app_train["SEX"] == 'N') | (app_train["SEX"] == ' '), "SEX"] = np.nan

# Convertir la columna "SEX" al tipo de dato str
app_train["SEX"] = app_train["SEX"].astype(str)

# Verificar los valores únicos actualizados en la columna "SEX"
sex_values = np.unique(app_train["SEX"], return_counts=True)
sex_counts = dict(zip(sex_values[0], sex_values[1]))
print(sex_counts)


{'F': 30805, 'M': 19130, 'nan': 65}


In [51]:
# Eliminar registros con el valor 'nan' en la columna "SEX"
app_train = app_train[app_train["SEX"] != 'nan']

# Verificar los valores únicos actualizados en la columna "SEX"
sex_values = app_train["SEX"].unique()
sex_counts = app_train["SEX"].value_counts().to_dict()
print(sex_counts)


{'F': 30805, 'M': 19130}


In [52]:
app_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49935 entries, 0 to 49999
Data columns (total 54 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID_CLIENT                       49935 non-null  int64  
 1   CLERK_TYPE                      49935 non-null  object 
 2   PAYMENT_DAY                     49935 non-null  int64  
 3   APPLICATION_SUBMISSION_TYPE     49935 non-null  object 
 4   QUANT_ADDITIONAL_CARDS          49935 non-null  int64  
 5   POSTAL_ADDRESS_TYPE             49935 non-null  int64  
 6   SEX                             49935 non-null  object 
 7   MARITAL_STATUS                  49935 non-null  int64  
 8   QUANT_DEPENDANTS                49935 non-null  int64  
 9   EDUCATION_LEVEL                 49935 non-null  int64  
 10  STATE_OF_BIRTH                  49935 non-null  object 
 11  CITY_OF_BIRTH                   49935 non-null  object 
 12  NACIONALITY                     

In [29]:
dtypes = {51: int, 52: int}

In [26]:
app_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 53 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID_CLIENT                       20000 non-null  int64  
 1   CLERK_TYPE                      20000 non-null  object 
 2   PAYMENT_DAY                     20000 non-null  int64  
 3   APPLICATION_SUBMISSION_TYPE     20000 non-null  object 
 4   QUANT_ADDITIONAL_CARDS          1273 non-null   float64
 5   POSTAL_ADDRESS_TYPE             20000 non-null  int64  
 6   SEX                             20000 non-null  object 
 7   MARITAL_STATUS                  20000 non-null  int64  
 8   QUANT_DEPENDANTS                20000 non-null  int64  
 9   EDUCATION_LEVEL                 20000 non-null  int64  
 10  STATE_OF_BIRTH                  20000 non-null  object 
 11  CITY_OF_BIRTH                   19990 non-null  object 
 12  NACIONALITY                     

In [24]:
dtypes = {51: int, 52: int}


### EDA